<a href="https://colab.research.google.com/github/ZubiaAhmed6345/Ai-Asssignments/blob/main/CREDICT_CARD_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import models,layers
from keras import regularizers

import io

In [2]:
from google.colab import files
uploaded = files.upload()

Saving creditcard.csv to creditcard.csv


In [3]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 1.9MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [8]:
df = pd.read_csv(io.BytesIO(uploaded['creditcard.csv']))
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,1.341262,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.058133,0.068653,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,-1.416907,-0.153826,-0.751063,0.167372,0.050144,-0.443587,0.002821,-0.611987,-0.045575,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,-0.619468,0.291474,1.757964,-1.323865,0.686133,-0.076127,-1.222127,-0.358222,0.324505,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,-0.705117,-0.110452,-0.286254,0.074355,-0.328783,-0.210077,-0.499768,0.118765,0.570328,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,1.017614,0.836390,1.006844,-0.443523,0.150219,0.739453,-0.540980,0.476677,0.451773,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [17]:
from sklearn.model_selection import train_test_split

nonfraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
nonfraud=nonfraud.sample(3*fraud.shape[0])
data = fraud.append(nonfraud, ignore_index=True)
data["Class"].value_counts()
x_data= data.drop(columns="Class", axis=0)
label = data["Class"]

train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [18]:
train_mean = train_data.mean(axis=0)
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [19]:
#model
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [21]:
model.fit(train_data, train_labels, epochs=100,validation_split = 0.3)

Epoch 1/100
31/31 [==============================] - 1s 11ms/step - loss: 1.0949 - accuracy: 0.5310 - val_loss: 0.6660 - val_accuracy: 0.7609
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 0.8867 - accuracy: 0.6787 - val_loss: 0.6401 - val_accuracy: 0.8696
Epoch 3/100
31/31 [==============================] - 0s 3ms/step - loss: 0.8029 - accuracy: 0.6766 - val_loss: 0.6220 - val_accuracy: 0.8720
Epoch 4/100
31/31 [==============================] - 0s 3ms/step - loss: 0.8268 - accuracy: 0.7053 - val_loss: 0.6146 - val_accuracy: 0.8382
Epoch 5/100
31/31 [==============================] - 0s 3ms/step - loss: 0.7687 - accuracy: 0.7361 - val_loss: 0.6026 - val_accuracy: 0.8237
Epoch 6/100
31/31 [==============================] - 0s 3ms/step - loss: 0.6868 - accuracy: 0.7571 - val_loss: 0.5924 - val_accuracy: 0.8140
Epoch 7/100
31/31 [==============================] - 0s 3ms/step - loss: 0.7197 - accuracy: 0.7389 - val_loss: 0.5784 - val_accuracy: 0.8116
Epoch 8/100


In [22]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 1ms/step - loss: 0.2736 - accuracy: 0.9594


In [24]:
accuracy

0.9593908786773682

In [25]:
loss

0.2735726237297058

In [27]:
x = model.predict(test_data).astype(dtype="u8")
a=x
b= np.array(test_labels)
a= a.reshape(1,a.shape[0])
np.size(a != b) - np.count_nonzero(a == b)

46

In [30]:
conf = tf.math.confusion_matrix(labels= test_labels, predictions=x )
conf

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[441,   2],
       [ 44, 104]], dtype=int32)>